In [ ]:
import pickle
import torch
from torch_geometric.data import Data,DataLoader
from functions_refactor import *
from pytorch_util import *
from torch.optim import Adam

In [ ]:
# model parameters
reuse = False
block = MEGNet_block
head = feedforwardHead_Update
data = '../Data/{}_data_ACSF_expand_PCA.pickle'
batch_size = 32
dim = 128
epochs = 20
clip = 0.4
layer1 = 3
layer2 = 3
factor = 2
lr = 1e-4

In [ ]:
train_dl,val_dl = get_data(data,batch_size)

In [ ]:
model = GNN_edgeUpdate(reuse,block,head,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')
paras = trainable_parameter(model)
opt = Adam(paras,lr=lr)
model, opt = amp.initialize(model, opt, opt_level="O1")
model,train_loss_list,val_loss_list,bestWeight = train_type(opt,model,epochs,train_dl,val_dl,paras,clip,\
                                                            UseAmp=True,AMP_clip=False)

In [ ]:
model = GNN_edgeUpdate(reuse,block,head,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')
paras = trainable_parameter(model)
opt = Adam(paras,lr=lr)
model, opt = amp.initialize(model, opt, opt_level="O0")
model,train_loss_list,val_loss_list,bestWeight = train_type(opt,model,epochs,train_dl,val_dl,paras,clip,\
                                                            UseAmp=False,AMP_clip=False)

In [ ]:
model = GNN_edgeUpdate(reuse,block,head,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')
paras = trainable_parameter(model)
opt = Adam(paras,lr=lr)
model, opt = amp.initialize(model, opt, opt_level="O2")
model,train_loss_list,val_loss_list,bestWeight = train_type(opt,model,epochs,train_dl,val_dl,paras,clip,\
                                                            UseAmp=False,AMP_clip=False)